Нарастим базу данных: соберём в неё таблицы **video_products** и **product_types**; добавим таблицу **slogans** (рекламный девиз или ударная фраза из фильма).

Записи из таблицы **video_products** будут ссылаться на слоганы через необязательную связь один к одному: у фильма может и не быть слогана.

![alt text](https://pictures.s3.yandex.net/resources/image_1710436594.png)

![alt text](https://pictures.s3.yandex.net/resources/image_1710436602.png)

![alt text](https://pictures.s3.yandex.net/resources/image_1710436610.png)

![alt text](https://pictures.s3.yandex.net/resources/S02_233_1685551291.png)

***
## Получаем связанные данные из таблиц

Попробуем получить из таблицы video_products все записи и для каждой из них — слоган, привязанный к ней через `slogan_id`. 

Для начала посмотрим, что вернёт запрос одновременно к двум таблицам:

In [1]:
import sqlite3
con = sqlite3.connect('db_video_type_slogan.sqlite')
cur = con.cursor()
results = cur.execute('''
    SELECT *
    FROM video_products, 
         slogans;
''')
for result in results:
    print(result)

con.close()

(1, 'Безумные Мелодии Луни Тюнз', 2, None, 1, "For Three Men The Civil War Wasn't Hell. It Was Practice!")
(1, 'Безумные Мелодии Луни Тюнз', 2, None, 2, "This isn't the movies anymore")
(1, 'Безумные Мелодии Луни Тюнз', 2, None, 3, 'Tonight on Murder She Wrote')
(1, 'Безумные Мелодии Луни Тюнз', 2, None, 4, "I'll be back")
(2, 'Весёлые мелодии', 2, None, 1, "For Three Men The Civil War Wasn't Hell. It Was Practice!")
(2, 'Весёлые мелодии', 2, None, 2, "This isn't the movies anymore")
(2, 'Весёлые мелодии', 2, None, 3, 'Tonight on Murder She Wrote')
(2, 'Весёлые мелодии', 2, None, 4, "I'll be back")
(3, 'Хороший, плохой, злой', 3, 1, 1, "For Three Men The Civil War Wasn't Hell. It Was Practice!")
(3, 'Хороший, плохой, злой', 3, 1, 2, "This isn't the movies anymore")
(3, 'Хороший, плохой, злой', 3, 1, 3, 'Tonight on Murder She Wrote')
(3, 'Хороший, плохой, злой', 3, 1, 4, "I'll be back")
(4, 'Последний киногерой', 3, 2, 1, "For Three Men The Civil War Wasn't Hell. It Was Practice!")
(4, 

Запрос вернул результирующую таблицу, включающую все столбцы обеих запрошенных таблиц БД:

`video_products.id` | `video_products.title` | `video_products.type_id` | `video_products.slogan_id` | `slogans.id` | `slogans.slogan_text`

В результирующую выборку включены все поля объединяемых таблиц; к каждой записи из video_products присоединена каждая запись из slogans. 

На математическом языке это называется «декартово произведение множеств» (или «прямое произведение множеств»).

Выглядит не очень практично, но ведь эту выборку можно отфильтровать!

Отфильтруем декартово произведение с помощью `WHERE`. Оставим только те строки, в  которых значение поля `slogan_id` в таблице **video_products** равно значению `id` в таблице **slogans** :

```sql
SELECT *
FROM video_products, 
     slogans
WHERE video_products.slogan_id = slogans.id;
```

![alt text](https://pictures.s3.yandex.net/resources/S02_13_2_1678066375.png)

```bash
(3, 'Хороший, плохой, злой', 3, 1, 1, "For Three Men The Civil War Wasn't Hell. It Was Practice!")
(4, 'Последний киногерой', 3, 2, 2, "This isn't the movies anymore")
(5, 'Она написала убийство', 4, 3, 3, 'Tonight on Murder She Wrote')
```


Для пущей красоты можно показать только полезные столбцы:

```sql
SELECT video_products.title,
       slogans.slogan_text
FROM video_products, 
     slogans
WHERE video_products.slogan_id = slogans.id;
```

```bash
('Хороший, плохой, злой', "For Three Men The Civil War Wasn't Hell. It Was Practice!")
('Последний киногерой', "This isn't the movies anymore")
('Она написала убийство', 'Tonight on Murder She Wrote')
```

***
## JOIN

Конструкция `FROM video_products, slogans` соединяет таблицы, а `WHERE` — фильтрует получившуюся выборку. 

По стандарту SQL92 принято отделять фильтрацию от условий соединения таблиц с помощью инструкции `JOIN` (англ. «соединять»):

```sql
-- Верни все поля
SELECT *
-- из таблицы video_products
FROM video_products
-- ...но перед этим присоедини записи из таблицы slogans так, чтобы
-- в записях значения полей video_products.slogan_id и slogans.id были равны.
JOIN slogans ON video_products.slogan_id = slogans.id;
```

Результат:

```bash
(3, 'Хороший, плохой, злой', 3, 1, 1, "For Three Men The Civil War Wasn't Hell. It Was Practice!")
(4, 'Последний киногерой', 3, 2, 2, "This isn't the movies anymore")
(5, 'Она написала убийство', 4, 3, 3, 'Tonight on Murder She Wrote')
```

Результат тот же, а запрос выглядит лучше:

* соединение и фильтрация разделены; это упрощает понимание запроса;

* условия соединения таблиц содержатся в блоке `ON`, это уменьшает вероятность ошибки.

В классическом SQL существует пять типов `JOIN`; SQLite поддерживает только три из них. Но с помощью ловкости и смекалки неподдерживаемые типы можно заменить другими запросами.

***
## Внутреннее пересечение: INNER JOIN

На первый тип `JOIN` вы уже полюбовались: в примере выше был как раз `INNER JOIN` (или просто `JOIN`, без титулов). 

Объединение таблиц через `INNER JOIN` можно представить схематически:

![alt text](https://pictures.s3.yandex.net/resources/S02_193_1685550377.png)

Запрос через `JOIN` можно сделать и к трём таблицам. Присоединим таблицу **product_types**:

```sql
SELECT video_products.title,
       slogans.slogan_text,
       product_types.title
FROM video_products
JOIN slogans ON video_products.slogan_id = slogans.id
JOIN product_types ON video_products.type_id = product_types.id; 
```

Теперь `JOIN` обработал сразу три таблицы!

```bash
('Хороший, плохой, злой', "For Three Men The Civil War Wasn't Hell. It Was Practice!", 'Фильм')
('Последний киногерой', "This isn't the movies anymore", 'Фильм')
('Она написала убийство', 'Tonight on Murder She Wrote', 'Сериал') 
```

***
## Левое внешнее соединение: LEFT OUTER JOIN

При обработке запроса `LEFT OUTER JOIN` объединяемые таблицы условно называют **«левая»** и **«правая»**. **«Левая»** — та, которая вызвана в блоке `FROM`, «правая» — та, что указана после ключевого слова `JOIN`. «Правых» таблиц может быть и несколько.

В этом запросе `OUTER` — необязательное слово. Можно использовать сокращённую запись: `LEFT JOIN`.

При выполнении запросов с `LEFT JOIN` возвращаются **все строки левой таблицы**. Данными из **правой таблицы** дополняются только те строки левой таблицы, для которых выполняются условия соединения, описанные после оператора `ON`. Для недостающих данных вместо строк правой таблицы вставляется `NULL`.

```sql
SELECT video_products.title,
       slogans.slogan_text
FROM video_products
LEFT JOIN slogans ON video_products.slogan_id = slogans.id; 
```

Результат запроса:

```bash
('Безумные мелодии Луни Тюнз', None)
('Весёлые мелодии', None)
('Хороший, плохой, злой', "For Three Men The Civil War Wasn't Hell. It Was Practice!")
('Последний киногерой', "This isn't the movies anymore")
('Она написала убийство', 'Tonight on Murder She Wrote') 
```

![alt text](https://pictures.s3.yandex.net/resources/S02_238_1678066511.png)

«Безумные мелодии Луни Тюнз» и «Весёлые мелодии» остались в выборке, поскольку `LEFT JOIN` возвращает **все записи левой таблицы без исключения**. Но у этих фильмов нет слогана, и вместо слогана вернулся `NULL` (`None` в переводе на язык Python).


***
## RIGHT OUTER JOIN

`RIGHT JOIN` — это такое же объединение, как и `LEFT JOIN`, но** выводятся все записи из правой таблицы**, а к ним добавляются только те данные из левой таблицы, в которых есть ключ объединения.

Напишем запрос через `RIGHT JOIN`:

```sql
SELECT video_products.title,
       product_types.title
FROM video_products
RIGHT JOIN product_types ON video_products.type_id = product_types.id; 
```

Результат запроса будет таким:

```bash
('Безумные мелодии Луни Тюнз', 'Мультсериал')
('Весёлые мелодии', 'Мультсериал')
('Последний киногерой', 'Фильм')
('Хороший, плохой, злой', 'Фильм')
('Она написала убийство', 'Сериал')
(None, 'Мультфильм') 
```

***
Даже если у вас в системе установлена новейшая, свежайшая версия оболочки sqlite — при использовании `RIGHT JOIN` из кода Python у вас всё равно может возникать ошибка `RIGHT and FULL OUTER JOINs are not currently supported`.

Дело в том, что python-библиотека sqlite3 не зависит от оболочки, установленной в системе, а использует собственные ресурсы. Поэтому при работе через консоль с оболочкой sqlite3 запросы с `RIGHT JOIN` и с `FULL OUTER JOIN` у вас будут выполняться на ура, а те же запросы, выполненные из кода Python, могут вернуть ошибку.
***

***
## FULL OUTER JOIN

При запросе **FULL (OUTER) JOIN** выводятся все записи из объединяемых таблиц. Те записи, у которых запрошенные значения совпадают, — выводятся парами, у остальных недостающее значение заменяется на `NULL` (Python выведет `None`).

Другими словами, **FULL JOIN == LEFT JOIN + RIGHT JOIN**.

В классическом SQL сработает такой запрос:

```sql
SELECT video_products.title,
       slogans.slogan_text
FROM video_products
FULL JOIN slogans ON video_products.slogan_id = slogans.id; 
```

Результат запроса:

```bash
(None, "I'll be back")
('Безумные мелодии Луни Тюнз', None)
('Весёлые мелодии', None)
('Она написала убийство', 'Tonight on Murder She Wrote')
('Последний киногерой', "This isn't the movies anymore")
('Хороший, плохой, злой', "For Three Men The Civil War Wasn't Hell. It Was Practice!") 
```

При возникновении такой ошибки можно пойти другим путём: выполнить два запроса `LEFT JOIN` и выполнить команду `UNION` — она объединяет данные из нескольких результирующих таблиц в одну:

```sql
SELECT video_products.title,
       slogans.slogan_text
FROM video_products
LEFT JOIN slogans ON video_products.slogan_id = slogans.id
UNION
SELECT video_products.title,
       slogans.slogan_text
FROM slogans
LEFT JOIN video_products ON video_products.slogan_id = slogans.id; 
```

***
## CROSS JOIN

Объединение таблиц через `CROSS JOIN` возвращает декартово произведение таблиц — каждая запись левой таблицы объединится с каждой записью правой. Параметр `ON` при запросах `CROSS JOIN` не применяется.

```sql
SELECT video_products.title,
       slogans.slogan_text
FROM video_products
CROSS JOIN slogans; 
```

Распечатаем результаты запроса:

```bash
('Безумные мелодии Луни Тюнз', "For Three Men The Civil War Wasn't Hell. It Was Practice!")
('Безумные мелодии Луни Тюнз', "This isn't the movies anymore")
('Безумные мелодии Луни Тюнз', 'Tonight on Murder She Wrote')
('Безумные мелодии Луни Тюнз', "I'll be back")
('Весёлые мелодии', "For Three Men The Civil War Wasn't Hell. It Was Practice!")
('Весёлые мелодии', "This isn't the movies anymore")
('Весёлые мелодии', 'Tonight on Murder She Wrote')
('Весёлые мелодии', "I'll be back")
('Хороший, плохой, злой', "For Three Men The Civil War Wasn't Hell. It Was Practice!")
('Хороший, плохой, злой', "This isn't the movies anymore")
('Хороший, плохой, злой', 'Tonight on Murder She Wrote')
('Хороший, плохой, злой', "I'll be back")
('Последний киногерой', "For Three Men The Civil War Wasn't Hell. It Was Practice!")
('Последний киногерой', "This isn't the movies anymore")
('Последний киногерой', 'Tonight on Murder She Wrote')
('Последний киногерой', "I'll be back")
('Она написала убийство', "For Three Men The Civil War Wasn't Hell. It Was Practice!")
('Она написала убийство', "This isn't the movies anymore")
('Она написала убийство', 'Tonight on Murder She Wrote')
('Она написала убийство', "I'll be back") 
```

На практике `CROSS JOIN` применяется не очень часто, но и он может быть полезен. Например, если в одной таблице сохранён список жидкостей, а во второй — список возможной тары для расфасовки, от маленькой баночки до цистерны. В выборке получим все возможные комбинации «жидкость — тара».

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

results = cur.execute('''
SELECT ice_cream.title,
       wrappers.title
FROM ice_cream
JOIN wrappers ON ice_cream.wrapper_id = wrappers.id
WHERE wrappers.title LIKE '%праздн%'; 
''')

for result in results:
    print(result)

con.close()

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

# Вместо многоточия напишите запрос, который вернёт нужные данные:
results = cur.execute('''
SELECT ice_cream.title,
       categories.slug,
       wrappers.title,
       MIN(ice_cream.price),
       AVG(ice_cream.price)
FROM ice_cream
JOIN categories ON ice_cream.category_id = categories.id
LEFT JOIN wrappers ON ice_cream.wrapper_id = wrappers.id
GROUP BY categories.id; 
''')

for result in results:
    print(result)

con.close()